In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, NoSuchFrameException

In [2]:
import pandas as pd
import numpy as np
import re
import time

In [3]:
def cafe_crawling2():
    description_state = False
    try:
        description_a_link = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt")
        description_a_link.click()
        description_state = True
    except Exception as ex:
        pass

    # 기본 카페 정보 크롤링
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    if not "대구" in address and not "경산" in address:
        return None

    cafe_name = soup.find('span', class_='Fc1rA').text
    cafe_type = soup.find('span', class_='DJJvD').text
    

    image_div = soup.find('div', {'id': 'ibu_1'})
    if image_div:
        style = image_div.get('style')
        background_image_url = re.search(r'url\((.*?)\)', style).group(1)
    else:
        background_image_url = ""
    
    # reviews_div = soup.find('div', class_='dAsGb')
    # starring = float(reviews_div.find('span', class_='PXMot LXIwF').text.strip()[2:]) if reviews_div.find('span', class_='PXMot LXIwF') else 0
    # if not reviews_div.find_all('span', class_='PXMot') or len(reviews_div.find_all('span', class_='PXMot')) < 3:
    #     return None
    # visitor_reviews = int(reviews_div.find_all('span', class_='PXMot')[1 if starring else 0].text.strip().split(" ")[1].replace(",", ""))
    # blog_reviews = int(reviews_div.find_all('span', class_='PXMot')[2 if starring else 1].text.strip().split(" ")[1].replace(",", ""))
    
    try:
        # 별점 요소 찾기
        star_rating_element = driver.find_element(By.CSS_SELECTOR, "div.dAsGb > span.PXMot.LXIwF")
        starring = float(star_rating_element.text.strip()[2:])
    except NoSuchElementException:
        # 별점 요소가 없는 경우
        starring = 0

    try:
        # 방문자 리뷰 요소 찾기
        visitor_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/visitor')]")
        visitor_reviews = int(visitor_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 방문자 리뷰 요소가 없는 경우
        visitor_reviews = 0
        
    try:
        # 블로그 리뷰 요소 찾기
        blog_review_element = driver.find_element(By.XPATH, "//span[@class='PXMot']/a[contains(@href, '/review/ugc')]")
        blog_reviews = int(blog_review_element.text.strip().split(" ")[1].replace(",", ""))
    except NoSuchElementException:
        # 블로그 리뷰 요소가 없는 경우
        blog_reviews = 0
    
    if visitor_reviews < 10:
        return None
    #PIbes > O8qbU > vV_z_ > zPfVt
    # description = soup.find_all('span', class_='zPfVt')[1].text if len(soup.find_all('span', class_='zPfVt')) > 1 else ""
    if description_state:
        description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > a > span.zPfVt").text
    else:
        try:
            description = driver.find_element(By.CSS_SELECTOR, "div.PIbes > div.O8qbU > div.vV_z_ > div.xHaT3 >  span.zPfVt").text
        except:
            description = ""
    address = soup.find('span', class_='LDgIH').text if soup.find('span', class_='LDgIH') else ""
    business_hours = soup.find('time').text if soup.find('time') else "" 
    phone_number = soup.find('span', class_='xlx7Q').text if soup.find('span', class_='xlx7Q') else ""

    # 키워드로 넘어가기
    keyword_tab_link = driver.find_element(By.XPATH, "//div[@class='flicking-camera']/a[.//span[contains(text(), '리뷰')]]")
    keyword_tab_link.click()
    
    keyword_state = False
    idx = 0
    while not keyword_state and idx < 5:
        try:
            keyword_list_link = WebDriverWait(driver, 0.5).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
            keyword_list_link.click()
            keyword_state = True
        except:
            for _ in range(5):
                keyword_tab_link.click()
                time.sleep(0.5)
        idx+=1
    
#     for _ in range(3):
#         try:
#             keyword_list_link = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "Tvx37")))
#             keyword_list_link.click()
#             keyword_state = True
#         except:
#             keyword_tab_link.click()
    
    if not keyword_state:
        return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", "", "", background_image_url]

    while True:
        try:
            keyword_more_link = driver.find_element(By.CLASS_NAME, 'Tvx37')
            keyword_more_link.click()
        except Exception as ex:
            break

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    keyword_number = int(driver.find_element(By.CSS_SELECTOR, "div._Wmab > em").text.replace(",", ""))

    keywords = {}
    ui_tags = soup.find('ui', class_='uNsI9')
    li_tags = soup.find_all('li', class_='nbD78')

    for li in li_tags:
        # 첫 번째 숫자를 int 형으로 변환
        text = li.find('span', class_='nWiXa').text.strip('"')
        numbers = re.findall(r'\d+', li.find('span', class_='TwM9q').text)
        number = int(numbers[0]) if numbers else None
        keywords[text] = number
    
    return [cafe_name, cafe_type, starring, visitor_reviews, blog_reviews, address, business_hours, phone_number, "", description, "", keywords, keyword_number, background_image_url]

In [4]:
cafe_search_list= [
                   "동성로", "대구 중앙로", "교동", "경북대병원역",
                    "동대구역", "대구 평화시장", "안심역",
                    "대구 광장코아", "만평역", "대구 서구청",
                    "명덕역", "대구 앞산",
                    "경북대 북문", "대구 복현오거리", "동천동 카페", "칠곡네거리",
                    "범어네거리", "수성유원지", "시지", "지산",
                    "두류공원", "용산", "월배", "계명대"
                    ]

In [ ]:
for cafe_name in ["대구 태전동", "수성유원지", "임당동", "경산 옥곡동"]:
    try:
        df = pd.DataFrame(columns=['cafe_name', "cafe_type", 'starring', 'visitor_review', 'blog_review', 'address', 'business_hours', 'phone_number', 'connection_site', 'description',
                                    "menu", "keyword", 'keyword_number', 'image_link'])
        # WebDriver 초기화
        driver = webdriver.Chrome() # Chrome 드라이버 사용
        driver.get(f"https://map.naver.com/p/search/{cafe_name} 카페") # 네이버 지도 접속

        # 결과 로드 대기
        time.sleep(3) # 페이지 로드를 위해 잠시 대기


        while True:
            # searchIframe으로 컨텍스트 전환 + div scroll 내리기
            driver.switch_to.frame("searchIframe")
            scroll_div = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "_pcmap_list_scroll_container")))
            for _ in range(5):
                driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', scroll_div)
                time.sleep(1)

            # 목록을 담고 있는 ul 태그를 찾고, 그 안의 모든 li 태그를 가져옴
            cafe_list = scroll_div.find_elements(By.TAG_NAME, "li")
            cafe_size = len(cafe_list)


            for item in cafe_list:
                try:
                    place_bluelink = item.find_element(By.CLASS_NAME, "place_bluelink")
                    # 부모의 부모인 a 태그 찾기 및 클릭
                    parent_a_tag = place_bluelink.find_element(By.XPATH, "../..")
                    parent_a_tag.click()

                    time.sleep(3)

                    # 필요한 작업 수행 (예: 상세 정보 수집)
                    driver.switch_to.default_content()
                    driver.switch_to.frame("entryIframe")
                    cafe_row = cafe_crawling2()
                    if cafe_row:
                        df.loc[len(df)] = cafe_row
                        print(cafe_row)

                    # 다음 카페로 이동하기 전에 searchIframe으로 다시 컨텍스트 전
                    driver.switch_to.default_content()
                    driver.switch_to.frame("searchIframe")

                except Exception as e:
                    print(e)
                    pass

            next_a_link = driver.find_elements(By.CLASS_NAME, "eUTV2")[1]
            aria_disabled = next_a_link.get_attribute("aria-disabled")
            # 'aria-disabled'가 'false'이면 a 태그 클릭
            if aria_disabled == "false":
                next_a_link.click()
            else:
                break

            # 원래 프레임으로 돌아감
            driver.switch_to.default_content()
    finally:
            df.to_csv(f"./new_cafe_data/{cafe_name}.csv", encoding='utf-8')


['마고노더니스_마고플레인태전점', '카페,디저트', 0, 2590, 366, '대구 북구 팔거천서로 35 태전동 1073-1번지', '09:00에 영업 시작', '0507-1365-5457', '', '', '', {'디저트가 맛있어요': 1581, '커피가 맛있어요': 1341, '인테리어가 멋져요': 1082, '음료가 맛있어요': 792, '매장이 청결해요': 719, '뷰가 좋아요': 587, '친절해요': 549, '주차하기 편해요': 504, '좌석이 편해요': 364, '특별한 메뉴가 있어요': 341, '대화하기 좋아요': 341, '사진이 잘 나와요': 333, '화장실이 깨끗해요': 318, '가성비가 좋아요': 150, '집중하기 좋아요': 92, '매장이 넓어요': 24, '빵이 맛있어요': 14, '종류가 다양해요': 6, '메뉴 구성이 알차요': 4, '오래 머무르기 좋아요': 4, '단체모임 하기 좋아요': 3, '음식이 맛있어요': 2, '아이와 가기 좋아요': 2, '룸이 잘 되어있어요': 2, '차분한 분위기에요': 1, '아늑해요': 1, '음식이 빨리 나와요': 1, '재료가 신선해요': 1, '선물하기 좋아요': 1, '야외 공간이 멋져요': 1, '비싼 만큼 가치있어요': 1, '특별한 날 가기 좋아요': 1, '건강한 맛이에요': 1, '차가 맛있어요': 1}, 2575, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230111_140%2F1673423410796uwzES_JPEG%2FKakaoTalk_20230111_164925724.jpg"']
['바루베이크샵', '카페,디저트', 4.67, 302, 131, '대구 북구 칠곡중앙대로54길 7 1층 바루베이크샵', '11:00에 영업 시작', '0507-1371-1489', '', '', '', {'디저트가

['디저트39 대구태전점', '카페', 0, 300, 33, '대구 북구 칠곡중앙대로69길 11 1층 디저트39', '11:30에 영업 시작', '053-312-3953', '', '', '', {'음료가 맛있어요': 117, '디저트가 맛있어요': 105, '커피가 맛있어요': 93, '친절해요': 87, '가성비가 좋아요': 66, '매장이 청결해요': 63, '특별한 메뉴가 있어요': 41, '대화하기 좋아요': 38, '좌석이 편해요': 34, '인테리어가 멋져요': 34, '화장실이 깨끗해요': 21, '집중하기 좋아요': 15, '사진이 잘 나와요': 14, '뷰가 좋아요': 9, '주차하기 편해요': 2, '아늑해요': 1, '양이 많아요': 1, '메뉴 구성이 알차요': 1, '룸이 잘 되어있어요': 1, '매장이 넓어요': 1, '오래 머무르기 좋아요': 1}, 295, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220831_144%2F16619091124229juIQ_JPEG%2FKakaoTalk_20220729_104756971_17.jpg"']
['투썸플레이스 대구태전점', '카페', 4.39, 444, 28, '대구 북구 칠곡중앙대로 393', '10:00에 영업 시작', '053-324-0019', '', '', '', {'커피가 맛있어요': 93, '디저트가 맛있어요': 85, '음료가 맛있어요': 54, '친절해요': 40, '대화하기 좋아요': 36, '매장이 청결해요': 32, '좌석이 편해요': 30, '집중하기 좋아요': 17, '인테리어가 멋져요': 15, '특별한 메뉴가 있어요': 11, '주차하기 편해요': 10, '화장실이 깨끗해요': 7, '뷰가 좋아요': 6, '사진이 잘 나와요': 5, '가성비가 좋아요': 5, '매장이 넓어요': 1, '오래 머무르기

['스프링캔 대구태전점', '카페,디저트', 0, 44, 16, '대구 북구 태전로 1 . 101동 106호(태전동, 태전역 더 뉴클래 스 상가)', '08:00에 영업 시작', '053-311-8608', '', '', '', {'가성비가 좋아요': 21, '친절해요': 20, '디저트가 맛있어요': 18, '커피가 맛있어요': 16, '음료가 맛있어요': 16, '매장이 청결해요': 16, '특별한 메뉴가 있어요': 8, '인테리어가 멋져요': 6, '대화하기 좋아요': 6, '좌석이 편해요': 4, '사진이 잘 나와요': 4, '주차하기 편해요': 3, '집중하기 좋아요': 1}, 44, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230116_251%2F1673832309569RNRay_PNG%2FKakaoTalk_20221230_140423260.png"']
['빽다방 대구보건대점', '테이크아웃커피', 4.63, 315, 19, '대구 북구 칠곡중앙대로69길 26', '08:00에 영업 시작', '053-311-3911', '', '', '', {'커피가 맛있어요': 87, '가성비가 좋아요': 65, '친절해요': 55, '음료가 맛있어요': 53, '디저트가 맛있어요': 17, '매장이 청결해요': 14, '대화하기 좋아요': 9, '특별한 메뉴가 있어요': 8, '집중하기 좋아요': 6, '인테리어가 멋져요': 5, '좌석이 편해요': 3, '사진이 잘 나와요': 2, '뷰가 좋아요': 2, '화장실이 깨끗해요': 1}, 200, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20200401_123%2F15857090642323hd4

['탑리터 태전점', '카페,디저트', 0, 130, 1, '대구 북구 영송로 66 1층', '07:00에 영업 시작', '0507-0289-3373', '', '', '', {'가성비가 좋아요': 60, '커피가 맛있어요': 43, '음료가 맛있어요': 41, '친절해요': 30, '디저트가 맛있어요': 14, '매장이 청결해요': 14, '특별한 메뉴가 있어요': 9, '대화하기 좋아요': 9, '좌석이 편해요': 7, '인테리어가 멋져요': 4, '집중하기 좋아요': 3, '화장실이 깨끗해요': 3, '뷰가 좋아요': 2, '음식이 빨리 나와요': 1, '사진이 잘 나와요': 1}, 130, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230323_267%2F16795623077979mJmS_JPEG%2F1679460782036.jpg"']
['혜민카페', '카페', 4.9, 61, 11, '대구 북구 태전로1길 13 혜민커피', '09:00에 영업 시작', '0507-1335-7549', '', '', '', {'친절해요': 22, '커피가 맛있어요': 20, '특별한 메뉴가 있어요': 9, '음료가 맛있어요': 8, '디저트가 맛있어요': 7, '매장이 청결해요': 6, '대화하기 좋아요': 6, '주차하기 편해요': 2, '가성비가 좋아요': 2, '좌석이 편해요': 1, '사진이 잘 나와요': 1}, 34, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230122_145%2F1674391650399DzmBf_JPEG%2Fimage.jpg"']
['파리바게뜨 대구과학대점', '베이커리', 4.59, 575, 10, '대구 북구 영송로

['커피앤번', '카페', 4.76, 94, 4, '대구 북구 칠곡중앙대로69길 20 커피앤번', '12:00에 영업 시작', '', '', '', '', {'커피가 맛있어요': 32, '친절해요': 21, '디저트가 맛있어요': 19, '음료가 맛있어요': 15, '가성비가 좋아요': 12, '인테리어가 멋져요': 7, '대화하기 좋아요': 6, '특별한 메뉴가 있어요': 5, '매장이 청결해요': 5, '좌석이 편해요': 3, '주차하기 편해요': 3, '사진이 잘 나와요': 3, '집중하기 좋아요': 3, '화장실이 깨끗해요': 2, '뷰가 좋아요': 2, '차분한 분위기에요': 1}, 72, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220302_269%2F1646196035845mKveW_JPEG%2F%25C4%25BF%25C7%25C7%25BE%25D8%25B9%25F8%25B7%25CE%25B0%25ED.jpg"']
['커피향을 담다', '카페,디저트', 0, 27, 0, '대구 북구 칠곡중앙대로46길 27', '09:00에 영업 시작', '053-242-0345', '', '', '', {'커피가 맛있어요': 14, '친절해요': 11, '대화하기 좋아요': 9, '매장이 청결해요': 8, '차분한 분위기에요': 6, '디저트가 맛있어요': 6, '음료가 맛있어요': 5, '특별한 메뉴가 있어요': 3, '집중하기 좋아요': 3, '좌석이 편해요': 1, '메뉴 구성이 알차요': 1, '주차하기 편해요': 1, '사진이 잘 나와요': 1, '화장실이 깨끗해요': 1}, 27, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F202304

...

['파리바게뜨 대구녹원점', '베이커리', 4.46, 611, 12, '대구 수성구 무학로 173', '09:00에 영업 시작', '0504-2015-4504', '', '', '', {'빵이 맛있어요': 286, '친절해요': 123, '매장이 청결해요': 77, '커피가 맛있어요': 37, '가성비가 좋아요': 33, '음료가 맛있어요': 28, '인테리어가 멋져요': 26, '특별한 메뉴가 있어요': 24, '주차하기 편해요': 15, '좌석이 편해요': 13, '대화하기 좋아요': 13, '뷰가 좋아요': 12, '화장실이 깨끗해요': 10, '사진이 잘 나와요': 6, '집중하기 좋아요': 4, '차분한 분위기에요': 1, '아늑해요': 1, '메뉴 구성이 알차요': 1}, 428, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220413_270%2F1649837147317UoMxL_PNG%2Fimage.png"']
['Stay Coffee', '카페', 4.39, 124, 36, '대구 수성구 수성로15길 12', '10:00에 영업 시작', '053-767-8582', '', '', '', {'커피가 맛있어요': 29, '친절해요': 17, '음료가 맛있어요': 16, '디저트가 맛있어요': 11, '대화하기 좋아요': 9, '인테리어가 멋져요': 8, '가성비가 좋아요': 6, '매장이 청결해요': 5, '좌석이 편해요': 4, '사진이 잘 나와요': 3, '집중하기 좋아요': 3, '특별한 메뉴가 있어요': 2, '화장실이 깨끗해요': 1, '뷰가 좋아요': 1}, 57, '']
['차우림', '카페,디저트', 0, 174, 44, '대구 수성구 지산로9길 22', '10:00에 영업 시작', '0507-1445-5524', '', '', '', {'커피가 맛있어요': 40, '친절해요

['프로나운 커피 쇼룸', '카페', 0, 10, 2, '대구 수성구 수성로 71 상가 C동 101호', '09:30에 영업 시작', '0507-1338-2829', '', '', '', '', '', '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230616_40%2F1686917005760GtjtF_PNG%2Fpronouncoffee_lab_20210320_%25282%2529.png"']
['투빈브레드', '베이커리', 4.44, 72, 7, '대구 수성구 수성로 135', '08:00에 영업 시작', '053-765-3777', '', '', '', {'빵이 맛있어요': 36, '가성비가 좋아요': 12, '친절해요': 11, '매장이 청결해요': 6, '특별한 메뉴가 있어요': 5, '커피가 맛있어요': 4, '음료가 맛있어요': 3, '뷰가 좋아요': 2, '주차하기 편해요': 1, '사진이 잘 나와요': 1, '집중하기 좋아요': 1, '대화하기 좋아요': 1}, 48, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20160909_63%2F1473387183962sDv83_JPEG%2F176857608862935_0.jpeg"']
['로운 ROUN', '카페,디저트', 0, 99, 61, '대구 수성구 들안로32길 91 1층', '매주 금요일 휴무', '0507-1371-6191', '', '', '', {'인테리어가 멋져요': 78, '친절해요': 59, '음식이 맛있어요': 42, '사진이 잘 나와요': 41, '특별한 메뉴가 있어요': 35, '매장이 청결해요': 30, '술이 다양해요': 24, '커피가 맛있어요': 16, '디저

['매스커피 황금점', '카페', 0, 143, 11, '대구 수성구 들안로32길 13 1층 매스커피 황금점', '07:30에 영업 시작', '053-764-1700', '', '', '', {'커피가 맛있어요': 81, '음료가 맛있어요': 48, '친절해요': 46, '매장이 청결해요': 33, '가성비가 좋아요': 32, '디저트가 맛있어요': 14, '대화하기 좋아요': 9, '인테리어가 멋져요': 8, '화장실이 깨끗해요': 7, '주차하기 편해요': 6, '특별한 메뉴가 있어요': 6, '좌석이 편해요': 5, '사진이 잘 나와요': 3, '집중하기 좋아요': 3, '뷰가 좋아요': 2}, 137, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20210405_151%2F1617627472655LpEWV_JPEG%2FvVbFOL-T3DgMjrK2Cx5_HAjN.jpg"']
['더리터 수성골드클래스점', '테이크아웃커피', 0, 27, 2, '대구 수성구 수성로 185 골드클래스 상가 101호', '08:00에 영업 시작', '070-8648-0887', '', '', '', {'커피가 맛있어요': 12, '음료가 맛있어요': 11, '가성비가 좋아요': 11, '친절해요': 10, '매장이 청결해요': 5, '특별한 메뉴가 있어요': 3, '화장실이 깨끗해요': 2, '좌석이 편해요': 1, '디저트가 맛있어요': 1}, 27, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220724_109%2F1658636083745JCCt5_JPEG%2F9772B41C-2D04-4D0B-9306-39D583D62293.jpeg"']
['에스프레소237

['카페인', '카페', 0, 65, 15, '대구 남구 앞산순환로 698 1층 카페인', '', '053-475-2286', '', '', '', {'커피가 맛있어요': 46, '디저트가 맛있어요': 20, '친절해요': 20, '음료가 맛있어요': 17, '인테리어가 멋져요': 15, '집중하기 좋아요': 13, '매장이 청결해요': 12, '가성비가 좋아요': 11, '특별한 메뉴가 있어요': 6, '좌석이 편해요': 3, '사진이 잘 나와요': 3, '화장실이 깨끗해요': 3, '대화하기 좋아요': 2, '뷰가 좋아요': 1}, 64, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20220429_156%2F1651239735021DbqGJ_JPEG%2F1651221483698-8.jpg"']
['올웨이즈누보', '카페,디저트', 0, 40, 25, '대구 수성구 수성로 185 수성 골드클래스 더 센텀 상가 201동 1층 103호', '10:00에 영업 시작', '0507-1478-1711', '', '', '', {'디저트가 맛있어요': 32, '커피가 맛있어요': 22, '특별한 메뉴가 있어요': 19, '음료가 맛있어요': 18, '친절해요': 14, '가성비가 좋아요': 9, '매장이 청결해요': 9, '사진이 잘 나와요': 7, '인테리어가 멋져요': 5, '주차하기 편해요': 2, '대화하기 좋아요': 2, '차분한 분위기에요': 1, '선물하기 좋아요': 1, '포장이 깔끔해요': 1, '집중하기 좋아요': 1}, 39, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20230222_1%2F1677052821574LVbeW_JPEG%2

['프로다카페', '카페,디저트', 4.46, 1340, 224, '경북 경산시 백천서로 82 프로다카페', '10:30에 영업 시작', '053-815-3722', '', '', '', {'커피가 맛있어요': 379, '뷰가 좋아요': 296, '주차하기 편해요': 241, '매장이 청결해요': 165, '음료가 맛있어요': 149, '친절해요': 137, '대화하기 좋아요': 134, '디저트가 맛있어요': 129, '인테리어가 멋져요': 123, '좌석이 편해요': 89, '화장실이 깨끗해요': 72, '사진이 잘 나와요': 58, '특별한 메뉴가 있어요': 37, '가성비가 좋아요': 36, '집중하기 좋아요': 33, '야외 공간이 멋져요': 1, '매장이 넓어요': 1}, 914, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%3A%2F%2Fldb-phinf.pstatic.net%2F20211108_183%2F1636351795808FIoHB_JPEG%2FjbYQopqgJlW3r0jN-2uLl27J.jpg"']
['더프레젠트', '카페,디저트', 4.59, 118, 126, '경북 경산시 선비길 21 카페더프레젠트', '12:00에 영업 시작', '053-814-5200', '', '', '', {'커피가 맛있어요': 22, '친절해요': 14, '음료가 맛있어요': 12, '인테리어가 멋져요': 11, '대화하기 좋아요': 11, '디저트가 맛있어요': 9, '매장이 청결해요': 9, '특별한 메뉴가 있어요': 6, '집중하기 좋아요': 6, '좌석이 편해요': 5, '사진이 잘 나와요': 5, '뷰가 좋아요': 3, '주차하기 편해요': 2, '화장실이 깨끗해요': 2, '가성비가 좋아요': 1}, 49, '"https://search.pstatic.net/common/?autoRotate=true&type=w560_sharpen&src=https%